In [188]:
import pandas as pd
import random
from scipy.sparse import coo_matrix, csr_matrix
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [112]:
interactions = pd.read_csv("D:\RecSys\syntetic_data.csv", usecols=['uid', 'item_id'])

In [113]:
df_weights = pd.DataFrame({"page_type": ["bought", "added to wishlist"],
                           'weight': [1, 1]})
df_weights

,page_type,weight
0,bought,1
1,added to wishlist,1


In [115]:
interactions2 = pd.DataFrame([],columns=['uid', 'item_id', 'page_type'])
lst_prods = []
for uid in range(0,len(interactions.uid)):
    lst_prods = list(filter(lambda x: x.isdigit(), [i.strip('[]\n') for i in interactions[interactions['uid']==uid]['item_id'].to_list()[0].split(' ')]))
    for prod in lst_prods:
        interactions2 = interactions2.append({'uid':uid, 'item_id':prod, 'page_type':random.sample(df_weights['page_type'].to_list(),1)[0]}, ignore_index=True)

In [286]:
products_dict = {}
new_products_dict = {}
n = 0
for i in set(interactions2['item_id']):
    products_dict[int(i)] = n
    new_products_dict[n] = int(i)
    n += 1

In [289]:
interactions2['new_item_id'] = [products_dict[int(item)] for item in interactions2['item_id']]
interactions2

,uid,item_id,page_type,new_item_id
0,0,11802,added to wishlist,295
1,0,11893,bought,113
2,0,11900,bought,135
3,0,11936,bought,511
4,0,12023,bought,493
...,...,...,...,...
4320,199,529589,added to wishlist,275
4321,199,574155,bought,284
4322,199,611127,bought,97
4323,199,645909,bought,479


In [290]:
data = pd.read_csv("D:\RecSys\products.csv")

In [291]:
weighted = interactions2.merge(df_weights, on="page_type")\
                  .groupby(["uid", "item_id", "new_item_id"])["weight"]\
                  .sum()\
                  .reset_index()

In [293]:
matrix = coo_matrix((weighted["weight"], (weighted["uid"], weighted["new_item_id"])), 
                    shape=(len(set(weighted["uid"])), len(set(weighted["new_item_id"]))))

In [294]:
class ColloborativeFiltering():
        def __init__(self, interactions):
            df_weights = pd.DataFrame({"page_type": ["bought", "added to wishlist"],
                           'weight': [1, 1]})
            #create table with weights and clean data
            interactions2 = pd.DataFrame([],columns=['uid', 'item_id', 'page_type'])
            lst_prods = []
            for uid in range(0,len(interactions.uid)):
                lst_prods = list(filter(lambda x: x.isdigit(), [i.strip('[]\n') for i in interactions[interactions['uid']==uid]['item_id'].to_list()[0].split(' ')]))
                for prod in lst_prods:
                    interactions2 = interactions2.append({'uid':uid, 'item_id':prod, 'page_type':random.sample(df_weights['page_type'].to_list(),1)[0]}, ignore_index=True)
            #create dict with new_products indexes
            self.products_dict = {}
            n = 0
            for i in set(interactions2['item_id']):
                self.products_dict[int(i)] = n
                n += 1
            #make new column with new item ids
            interactions2['new_item_id'] = [self.products_dict[int(item)] for item in interactions2['item_id']]
            #make new column with weights
            weighted = interactions2.merge(df_weights, on="page_type")\
                  .groupby(["uid", "item_id", "new_item_id"])["weight"]\
                  .sum()\
                  .reset_index()
            #create coordinate matrix for users and items
            self.matrix = coo_matrix((weighted["weight"], (weighted["uid"], weighted["new_item_id"])), 
                    shape=(len(set(weighted["uid"])), len(set(weighted["new_item_id"]))))
            
        def get_rec_U2I(self, users_products):
            #create user_interactions array
            usr_int_arr = np.zeros(matrix.shape[1])
            for i in users_products:
                usr_int_arr[self.products_dict[i]]=1
            #косинусная близость между пользователем и остальными пользователями
            metrics = cosine_similarity([usr_int_arr], self.matrix).reshape(-1, 1)
            #умножаем  все взаимодействия каждого пользователя с товарами на косинусную близость с этим пользователем
            rates = csr_matrix.multiply(self.matrix.copy(), metrics)
            #складываем коэффициенты для каждого товара по всем пользователям и домножаем на 0 те товары, которые уже есть в выборке
            total_rate = (1 - usr_int_arr.astype(bool)) * np.array(np.sum(rates, axis=0))[0]
            return total_rate


In [295]:
CB = ColloborativeFiltering(interactions)

In [334]:
users_products = [96087,114517,70110,110624]

In [335]:
rates = CB.get_rec_U2I(users_products)

In [336]:
prod_metrics = {}
for i in range(0,len(rates)):
    prod_metrics[i] = rates[i]
sorted_list = sorted(prod_metrics.items(), key=lambda x: x[1], reverse=True)
sorted_list[:15]

[(474, 2.375321911672758),
 (510, 2.083819950792788),
 (285, 1.9644188726100726),
 (588, 1.91639016195043),
 (224, 1.849511111203743),
 (437, 1.7876517992850596),
 (343, 1.6805506449289511),
 (75, 1.587702975840425),
 (90, 1.5739502867511457),
 (326, 1.4856409032244593),
 (102, 1.4811026176626199),
 (462, 1.4785940307224288),
 (148, 1.4758995769654355),
 (454, 1.462154245659538),
 (580, 1.3917187776349795)]

In [337]:
user_df = pd.DataFrame([], columns=['name','article'])
for product in users_products:
    user_df = user_df.append({'name':data[data['article']==product]['name'],'article':product}, ignore_index=True)
user_df

,name,article
0,"273 Кукла Роралай CAVE CLUB Name: name, dty...",96087
1,287 Набор с куклой Ася Семья TOYSLAB Name: ...,114517
2,370 LEGO City Fire Лесные пожарные арт. 602...,70110
3,371 Lego 60279 пожарный автомобиль Name: na...,110624


In [338]:
results = pd.DataFrame([],columns=['name','article','metric'])
for i in sorted_list[:15]:
    results = results.append({'name' : data[data['article']==new_products_dict[i[0]]]['name'].to_list()[0], 'article' : int(new_products_dict[i[0]]), 'metric' : i[1]}, ignore_index=True)

In [339]:
results

,name,article,metric
0,Lego 60304 перекрёсток,110621,2.375322
1,Lego 60287 трактор,110601,2.083820
2,Lego 10953 Единорог,110607,1.964419
3,Lego 71381 кусалкин на цепи,110620,1.916390
4,LEGO City Fire Пожар в бургер-кафе арт. 60214,659801,1.849511
5,Набор кукла с мебелью Miliya Valori,119664,1.787652
6,Кукла Disney Frozen Холодное сердце 2 в ассорт...,111843,1.680551
7,Enchantimals базовая кукла со зверушкой в ассорт,608874,1.587703
8,Barbie кукла космонавт,112553,1.573950
9,Кукла Эмберли CAVE CLUB,96090,1.485641
